In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_curve
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv("./labeled.csv", sep=",")

In [4]:
df.shape

(14412, 2)

In [5]:
df.tail(5)

,comment,toxic
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0
14411,До сих пор пересматриваю его видео. Орамбо кст...,0.0


In [6]:
df["toxic"] = df["toxic"].apply(int)

In [7]:
df.tail(5)

,comment,toxic
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1
14411,До сих пор пересматриваю его видео. Орамбо кст...,0


In [8]:
df["toxic"].value_counts()

toxic
0    9586
1    4826
Name: count, dtype: int64

In [9]:
for c in df[df["toxic"] == 1]["comment"].tail(5):
    print(c)


ЖЕНЩИНА это ВЕНЕЦ ТВОРЕНИЯ, помните это ваньки! Вы все не стоите одной женской какашки, лошки! Женщины правят цивилизацией! На колени хуемрази! Феминизм победил! КНН

Сейчас все авиакомпании мира вместе со специалистами гугл карт смотрят на казахов и думают ну что за дебилы ну еп твое мать.

Вонючий совковый скот прибежал и ноет. А вот и сторонник демократии и свободы слова закукарекал.

А кого любить? Гоблина тупорылого что-ли? Или какую-нибудь продажную суку из демшизы? Нет уж, спасибо не надо.

КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ НЕТ ОБСУЖДЕНИЯ ПОЛИТИКИ. СВОБОДНОЕ ОБЩЕНИЕ ЭТО В b. ЭТО ТОЖЕ САМОЕ ЕСЛИ Я НА ДОСКЕ О ПОЛИТИКЕ СОЗДАМ ТРЕД О ТОМ КАК ПЛОХО БЕЗ ТЯНОЧКИ. ТАК ЧТО УЕБЫВАЙТЕ В Б ИЛИ НВР СО СВОИМ ЧАТИКОМ ПРЕСТАРЕЛЫХ ГОМОСЕКОВ!



In [10]:
for c in df[df["toxic"] == 0]["comment"].tail(5):
    print(c)


ТАСС, 21 марта. Премьер-министр Новой Зеландии Джасинда Ардерн ввела в четверг запрет на оборот и эксплуатацию штурмовых и полуавтоматических винтовок на территории страны. Об этом говорится в официальном заявлении, распространенном в четверг и опубликованном на сайте новозеландского правительства. В понедельник, через 72 часа после ужасного террористического акта в Крайстчерче кабинет министров согласился пересмотреть закон. Теперь, через шесть дней после этого нападения, мы объявляем о запрете на все полуавтоматические и штурмовые винтовки военного образца в Новой Зеландии , - говорится в заявлении. Все виды полуавтоматического оружия, которые были использованы во время теракта в пятницу, 15 марта, будут запрещены . Отмечается, что под запрет также попадут приспособления, с помощью которых можно переделать оружие в полуавтоматическое военного образца, и магазины большой емкости. Помимо этого, кабинет министров поручил разработать схему выкупа оружия подобного типа у его владельцев. И

In [11]:
train_df, test_df = train_test_split(df, test_size=500)

In [12]:
test_df.shape

(500, 2)

In [13]:
test_df["toxic"].value_counts()

toxic
0    335
1    165
Name: count, dtype: int64

In [14]:
train_df["toxic"].value_counts()

toxic
0    9251
1    4661
Name: count, dtype: int64

In [15]:
sentence_example = df.iloc[2]["comment"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]



In [16]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Собаке - собачья смерть

-----------------
Токены: ['Собаке', '-', 'собачья', 'смерть']
-----------------
Токены без пунктуации: ['Собаке', 'собачья', 'смерть']
-----------------
Токены без пунктуации и стоп слов: ['Собаке', 'собачья', 'смерть']
-----------------
Токены после стемминга: ['собак', 'собач', 'смерт']
-----------------


In [17]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [27]:
tokenize_sentence(sentence_example)

['собак', 'собач', 'смерт']

In [28]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))


In [29]:
features = vectorizer.fit_transform(train_df["comment"])

In [30]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df["toxic"])

LogisticRegression(random_state=0)

In [31]:
model.predict(features[0])

array([0], dtype=int64)

In [32]:
train_df["comment"].iloc[0]

'В чем проблема с онлайн чеком? Если по честному отработать и заплатить налог? Только что муторно, но мне надо чек на 1000, ок переводи мне 1000, получай чек и потом я тебе верну обратно 700, а 300 это на налог обнал гешефт.\n'

In [33]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0))
]
)

In [34]:
model_pipeline.fit(train_df["comment"], train_df["toxic"])

D:\ProgramFilesi\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x0000020E5B951080>)),
                ('model', LogisticRegression(random_state=0))])

In [35]:
model_pipeline.predict(["Как ты себя чувствуешь?"])

array([0], dtype=int64)

In [36]:
model_pipeline.predict(["Гореть вам всем в аду!"])

array([1], dtype=int64)

In [38]:
precision_score(y_true=test_df["toxic"], y_pred=model_pipeline.predict(test_df["comment"]))

0.9069767441860465

In [40]:
recall_score(y_true=test_df["toxic"], y_pred=model_pipeline.predict(test_df["comment"]))

0.7090909090909091

In [41]:
prec, rec, thresholds = precision_recall_curve(y_true=test_df["toxic"], probas_pred=model_pipeline.predict_proba(test_df["comment"])[:, 1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(rec, prec, color='purple')

#add axis labels to plot
ax.set_title('Precision-Recall Curve')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')

#display plot
plt.show() 